## Angry Tweets

Import używanych bibliotek i kompilacja zastosowanych wyrażeń regularnych.

In [1]:
import re
import pandas as pd
import nltk
import string
from collections import Counter
from scipy.sparse import csr_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score

RE_SPACES = re.compile("\s+")
RE_HASHTAG = re.compile("[@#][_a-z0-9]+")
RE_EMOTICONS = re.compile("(:-?\))|(:p)|(:d+)|(:-?\()|(:/)|(;-?\))|(<3)|(=\))|(\)-?:)|(:'\()|(8\))")
RE_HTTP = re.compile("http(s)?://[/\.a-z0-9]+")
RE_NUM = re.compile("[0-9]+")

Wczytanie danych treningowych w celu utworzenia słownika. Wyrzucam wszystkie tweety z tekstem 'Not Available', ponieważ oceniane są różnie i wprowadzają szum. Można próbować zastąpić je kopiując losowy tweet z danej klasy jednak nie wniesie to nowej jakości do danych.

In [2]:
tweets = pd.read_csv("train.csv", sep=",", index_col=None, na_values = 'Not Available').iloc[:,2]
tweets = tweets.dropna()

Deklaracja metody obsługującej przetworzenie encji html na poprawne znaki tekstowe oraz deklaracja tokenizatora dla tweetów.

In [3]:
class BeforeTokenizationNormalizer():
    @staticmethod
    def normalize(text):
        text = text.strip().lower()
        text = text.replace('&nbsp;', ' ')
        text = text.replace('&lt;', '<')
        text = text.replace('&gt;', '>')
        text = text.replace('&amp;', '&')
        text = text.replace('&pound;', u'£')
        text = text.replace('&euro;', u'€')
        text = text.replace('&copy;', u'©')
        text = text.replace('&reg;', u'®')
        return text

class TweetTokenizer():
    @staticmethod
    def tokenize(text):
        tokens = text.split()
        i = 0
        while i < len(tokens):
            token = tokens[i]
            match = re.search(RE_HASHTAG, token) or re.search(RE_EMOTICONS, token) or re.search(RE_HTTP, token)
            if match is not None:
                del tokens[i]
                i -= 1
                match = None
            else:
                del tokens[i]
                tokens[i:i] = nltk.word_tokenize(token)
            i += 1
            
        porter = nltk.PorterStemmer()
        tokens2 = []
        for token in tokens:
            token2 = porter.stem(token)
            if token2 == "n't":
                token2 = "not"
            if token2 not in string.punctuation:
                tokens2.append(token2)
        return tokens2

Stworzenie słownika dla tweetów pomijającego znaki interpunkcyjne i cyfry.

In [4]:
words = Counter()

for i in range(len(tweets)):
    tweet = BeforeTokenizationNormalizer.normalize(tweets.iat[i])
    words.update(TweetTokenizer.tokenize(tweet))

words2 = dict(words)
for word in words:
    for letter in word:
        if letter in string.punctuation:
            del words2[word]
            break

words3 = dict(words2)
for i in words2:
    if re.search(RE_NUM,i):
        del words3[i]
words = Counter(words3)

Opisanie tweetów w reprezentacji bag of words i zamknięcie ich w macierzy CSR (skompresowana wierszami postać macierzy rzadkiej) w celu zmniejszenia jej rozmiaru.

In [5]:
def create_bow(documents, features):
    row = []
    col = []
    data = []

    labels = []

    for i in range(len(documents)):
        tweet = BeforeTokenizationNormalizer.normalize(documents.iat[i, 2])
        label = documents.iloc[i, 1]
        tweet_tokens = TweetTokenizer.tokenize(tweet)

        labels.append(label)
        for token in set(tweet_tokens):
            if token not in features:
                continue
            row.append(i)
            col.append(features[token])
            data.append(1)
    return csr_matrix((data, (row, col)), shape=(len(documents), len(features))), labels

def create_bow2(documents, features):
    row = []
    col = []
    data = []
    for i in range(len(documents)):
        tweet = BeforeTokenizationNormalizer.normalize(documents.iat[i, 1])
        tweet_tokens = TweetTokenizer.tokenize(tweet)

        for token in set(tweet_tokens):
            if token not in features:
                continue
            row.append(i)
            col.append(features[token])
            data.append(1)
    return csr_matrix((data, (row, col)), shape=(len(documents), len(features)))

Stworzenie klasyfikatora i ocena tweetów testowych. Minimalna liczba wystąpień słowa została dobrana eksperymentalnie i wynosi 50. 

In [6]:
min_word_count = 50

train_tweets = pd.read_csv("train.csv", sep=",", na_values = 'Not Available')
train_tweets = train_tweets.dropna()
common_words = list([k for k, v in words.most_common() if v > min_word_count])

feature_dict = {}
for word in common_words:
    feature_dict[word] = len(feature_dict)

X_train, y_train = create_bow(train_tweets, feature_dict)
list_of_labels = list(set(y_train))
classifier = RandomForestClassifier(criterion = "entropy", n_estimators=50, n_jobs=-1, random_state=111)
classifier.fit(X_train, y_train)

test_tweets = pd.read_csv("test.csv", sep=",", na_values = 'nan', dtype = str)
test_tweets = test_tweets.dropna()
X_test = create_bow2(test_tweets, feature_dict)
predicted = classifier.predict(X_test)

f = open("sub.csv", 'w')
f.write("Id,Category\n")
for i in range(len(test_tweets)):
    f.write(test_tweets.iat[i,0]+','+predicted[i]+'\n')
f.close()